In [ ]:
# Mean Absolute Error
# Mean Squared Error
# Root Mean Square Error
# R Squared Values

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('linear_regression').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/22 17:29:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
from pyspark.ml.regression import LinearRegression

In [12]:
data = spark.read.csv(
    path='data/Ecommerce_Customers.csv', 
    inferSchema=True, 
    header=True,
    multiLine=True)
print(data.count(), len(data.columns))

500 8


In [13]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg. Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [14]:
data.show(5)

+--------------------+--------------------+----------------+-------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg. Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+-------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank Tunnel\...|          Violet|  34.49726772511229|12.655651149166752| 39.57766801952616|   4.082620632952961|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.926272026360156|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque| 33.000914755642675|11.330278057777512| 37.11059744212085|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug.

In [16]:
data.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank Tunnel\nWrightmouth, MI 82180-9605', Avatar='Violet', Avg. Session Length=34.49726772511229, Time on App=12.655651149166752, Time on Website=39.57766801952616, Length of Membership=4.082620632952961, Yearly Amount Spent=587.9510539684005)]

### Test

In [18]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [27]:
data = data.withColumnRenamed('Avg. Session Length', 'Avg Session Length')

In [28]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [46]:
assembler = VectorAssembler(inputCols=[
    'Avg Session Length',
    'Time on App',
    'Time on Website',
    'Length of Membership'],
    outputCol='features')

In [47]:
output = assembler.transform(data)

In [48]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank Tunnel\nWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.655651149166752, Time on Website=39.57766801952616, Length of Membership=4.082620632952961, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [49]:
final_data = output.select('features', 'Yearly Amount Spent')
final_data.show(5)

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352178|
|[33.3306725236463...|  599.4060920457634|
+--------------------+-------------------+
only showing top 5 rows



In [50]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])
print(train_data.count())
print(test_data.count())

345
155


In [51]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                345|
|   mean| 499.76573920502426|
| stddev|  77.58906780971792|
|    min| 298.76200786180766|
|    max|  765.5184619388372|
+-------+-------------------+



In [52]:
lr = LinearRegression(labelCol='Yearly Amount Spent')
lr_model = lr.fit(train_data)
test_results = lr_model.evaluate(test_data)

23/02/22 18:11:37 WARN Instrumentation: [e4cc0fcc] regParam is zero, which might cause numerical instability and overfitting.


In [53]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|  9.926889371214713|
| -5.561824866427116|
|-17.805180151501986|
| -5.073188043216533|
|  3.677765944850421|
|-13.316848385037247|
|  8.529861978153974|
| -4.129351080920571|
|  21.59813884367759|
|  2.582125317638088|
|   17.8743310681989|
|  3.533400052791251|
|0.08347939048059061|
| -5.209301307592682|
| -5.103457994471057|
| 1.8010360850790335|
|-15.173216350709481|
|-2.7631839755546252|
| 5.3105561427686325|
| -7.025654440576545|
+-------------------+
only showing top 20 rows



In [54]:
print(test_results.rootMeanSquaredError)
print(test_results.r2)
print(test_results.meanSquaredError)
print(test_results.meanAbsoluteError)

10.520125786267563
0.9839378638072758
110.6730465588917
8.288809999636962


In [55]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|   499.314038258591|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388372|
+-------+-------------------+



### Unlabeled

In [44]:
unlabeled_data = test_data.select('features')
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.4925366965401...|
|[30.9716756438877...|
|[31.0662181616375...|
|[31.1239743499119...|
|[31.1695067987115...|
|[31.3091926408918...|
|[31.4252268808548...|
|[31.4474464941278...|
|[31.5257524169682...|
|[31.5261978982398...|
|[31.5741380228732...|
|[31.6005122003032...|
|[31.6253601348306...|
|[31.6548096756927...|
|[31.6739155032749...|
|[31.7242025238451...|
|[31.8209982016720...|
|[31.8648325480987...|
|[31.9096268275227...|
|[31.9120759292006...|
+--------------------+
only showing top 20 rows



In [45]:
predictions = lr_model.transform(unlabeled_data)
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.4925366965401...| 282.4712457199143|
|[30.9716756438877...|494.63860975689295|
|[31.0662181616375...|448.93329320767384|
|[31.1239743499119...|  486.947053839765|
|[31.1695067987115...| 427.3565308022931|
|[31.3091926408918...|432.72071783993374|
|[31.4252268808548...| 530.7667186547619|
|[31.4474464941278...|418.60274209522373|
|[31.5257524169682...| 443.9656268098817|
|[31.5261978982398...| 409.0945261923375|
|[31.5741380228732...| 544.4092721605863|
|[31.6005122003032...| 479.1728514910976|
|[31.6253601348306...| 376.3369007569241|
|[31.6548096756927...| 475.2634237275487|
|[31.6739155032749...|475.72506790988024|
|[31.7242025238451...|503.38788728796027|
|[31.8209982016720...| 424.6752810132136|
|[31.8648325480987...| 439.8912804768133|
|[31.9096268275227...| 563.4460356732396|
|[31.9120759292006...|387.53471630570766|
+--------------------+------------